In [73]:
import pickle
import pandas as pd
from tqdm import tqdm
import requests

In [4]:
dis = pickle.load(open('1001.pkl', 'rb'))

In [7]:
print(len(dis))
dis[::500]

5503


['List.docx',
 'Part 01/Frank_Sinatra_-_In_the_Wee_Small_Hours-1991-OBSERVER/14_Frank_Sinatra_-_Dancing_on_the_Ceiling.-OBSERVER.mp3',
 'Part 01/Monks_-_Black_Monk_Time-1966-OBSERVER/02_Monks_-_Shut_Up.-OBSERVER.mp3',
 'Part 01/The_Who_-_Sell_Out-2CD-1967-OBSERVER/122_The_Who_-_Girlss_Eyes-OBSERVER.mp3',
 'Part 03/Janis_Joplin_-_Pearl-VINYL-1971-OBSERVER/02_Janis_Joplin_-_Cry_Baby-OBSERVER.mp3',
 'Part 03/Sly_and_the_Family_Stone_-_Theres_A_Riot_Goin_on-1971-OBSERVER/08_Sly_and_the_Family_Stone_-_Time-OBSERVER.mp3',
 'Part 04/David_Bowie_-_Heroes-1977-OBSERVER/01_David_Bowie_-_Beauty_and_the_Beast-OBSERVER.mp3',
 'Part 04/Neu_-_Neu_75-1975-OBSERVER/05_Neu_-_E-Musik-OBSERVER.mp3',
 'Part 04/Wire_-_Pink_Flag-1977-OBSERVER/24_Wire_-_Love_Aint_Polite-OBSERVER.mp3',
 'Part 05/Heaven 17 -_Penthouse_and_Pavement-REMASTERED-1981-OBSERVER/12_Penthouse_and_Pavement_-_Im_Your_Money_(12_Version).-OBSERVER.mp3',
 'Part 05/The_Jam_-_All_Mod_Cons_and_Sound_Affects-1996-OBSERVER/03_The_Jam_-_Mr._Clean

In [9]:
EXTS = {'mp3', 'flac', 'm3u', 'm4a', 'wma'}

In [10]:
exts = sorted({n.split('.')[-1].lower() for n in dis})
', '.join(exts)

'docx, ini, jpg, mp3, nfo, txt'

In [56]:
_music = filter(lambda n: n.split('.')[-1].lower() in EXTS, dis)
_music = map(lambda p: '/'.join(p.split('/')[1:]).replace('_', ' '), _music)

music = []
for m in _music:
    m = m.replace('-OBSERVER', '').strip('.mp3')
    ba, sng = m.split('/')
    band, *alb = ba.split('-')
    band = band.strip()
    album = '-'.join(alb)
    
    song = sng.split('-')
    if band in song[0]:
        song = '-'.join(song[1:])
    else:
        song = '-'.join(song)
    
    music.append((band, album.strip(), song.strip()))

In [57]:
len(music)

5320

In [58]:
{len(m) for m in music}

{3}

In [59]:
music

[('Astrud Gilberto', 'Beach Samba-1967', 'Stay'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Misty Roses'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'The Face I Love'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'A Banda (Parade)'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Oba Oba'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Canoeiro'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'I Had the Craziest Drea'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Bossa Na Praia (Beach Samba)'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'My Foolish Heart'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Dia Das Rosas (I Think of You)'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'You Didnt Have to Be So Nice'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Nao Bate O Corocao'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Goodbye Sadness'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Call Me'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Heres that Rainy Day'),
 ('Astrud Gilberto', 'Beach Samba-1967', 'Tu Meu Delirio'),
 ('A

In [64]:
from collections import defaultdict

bands = defaultdict(dict)
for band, album, song in music:
    bands[band].setdefault(album, [])
    bands[band][album].append(song)

In [68]:
sorted(bands.keys())

['10CC',
 'ABBA',
 'ABC',
 'AC',
 'ACDC',
 'Abba',
 'Adam and the Ants',
 'Aerosmith',
 'Air',
 'Al Green',
 'Alanis Morissette',
 'Alice Cooper',
 'Ash',
 'Astrud Gilberto',
 'B',
 'B.B. King',
 'Bad Company',
 'Bauhaus',
 'Bee Gees',
 'Bert Jansch',
 'Big Star',
 'Bill Evans Trio',
 'Billie Holiday',
 'Billy Joel',
 'Black Flag',
 'Black Sabbath',
 'Blondie',
 'Bob Dylan',
 'Bob Marley',
 'Bob Marley and the Wailers',
 'Bobby Womack',
 'Booker T. and the M.G.S',
 'Boston',
 'Brian Eno',
 'Brian Eno and David Byrne',
 'Bruce Springsteen',
 'Buck Owens',
 'Buddy Holly',
 'Buffalo Springfield',
 'Burning Spear',
 'Buzzcocks',
 'Can',
 'Captain Beefheart',
 'Carole King',
 'Charles Mingus',
 'Cheap Trick',
 'Chic',
 'Circle Jerks',
 'Count Basie',
 'Country Joe and the Fish',
 'Cream',
 'Curtis Mayfield',
 'David Ackles',
 'David Bowie',
 'David Crosby',
 'Dead Kennedys',
 'Deep Purple',
 'Dennis Wilson',
 'Devo',
 'Dexys Midnight Runners',
 'Dion',
 'Dire Straits',
 'Dolly Parton',
 'Do

---

In [75]:
API_BASE = 'http://127.0.0.1:8000'
ADD_BAND_URL = f'{API_BASE}/music/bands'
ADD_ALBUM_URL = f'{API_BASE}/music/albums'

In [76]:
for band, albums in tqdm(bands.items()):
    res = requests.post(ADD_BAND_URL, data={'band': band}).json()
    band_id = res['band_id']
    for album, songs in albums.items():
        res = requests.post(ADD_ALBUM_URL, json={
            'album': album, 
            'band_id': band_id,
            'songs': songs,
        }).json()

100%|██████████████████████████████| 308/308 [00:49<00:00,  6.28it/s]
